# Docker Host
Sử dụng lệnh docker-machine để tạo ra cá máy ảo Docker, quản lý docker Machine trên máy local

In [41]:
!docker-machine ls

NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS


In [43]:
# Firstly created node is manager/leader
!docker-machine create --driver virtualbox --virtualbox-memory "2048" --virtualbox-hostonly-cidr 192.168.56.1/21 vps1

Docker machine "vps1" already exists


In [ ]:
!docker-machine create --driver virtualbox vps2

In [ ]:
!docker-machine create --driver virtualbox vps3

In [ ]:
!docker-machine rm vps1

In [ ]:
!docker-machine stop vps1 vps2

In [ ]:
!docker-machine start vps1 vps2 

In [17]:
!docker-machine ls

NAME   ACTIVE   DRIVER       STATE     URL   SWARM   DOCKER    ERRORS
vps1   -        virtualbox   Stopped                 Unknown   
vps2   -        virtualbox   Stopped                 Unknown   
vps3   -        virtualbox   Stopped                 Unknown   


In [ ]:
!docker-machine ip vps1

In [ ]:
!docker-machine scp -r data/ vps1:/home/

In [ ]:
!docker-machine ssh vps1

# Docker Swarm

<b>Khi một cụm các máy (cluster) cùng chạy Docker Engine tập hợp lại kết nối với nhau nó hình thành nên một Docker Swarm, mỗi máy này có thể là một máy vật lý thật, VPS hay là máy ảo tự tạo.

<b> Doi thu truc tiep cua docker swarm la Kubernetes

In [ ]:
# Inside vps1 (manager/leader)
!docker swarm init --advertise-addr=192.168.99.117

!docker info

!docker node ls

!docker swarm join-token worker

!docker node rm 'id'

In [ ]:
# Inside vps2, vps3 (worker)
!docker swarm join --token ...

!docker swarm leave

<b>Docker stack</b><br>là một command được nhúng vào Docker CLI, giúp quản lý một vùng chứa (cluster) của docker containter thông qua docker swarm. <br>

Lệnh docker stack để quản lý các dịch vụ chạy trên swarm, các thành phần của dịch vụ đó được mô tả trong file docker-compose.yml

<b>Mạng Overlay trên Docker Swarm</b><br>
Mạng mà các container chạy trên swarm kết nối vào là loại mạng overlay, mạng này kết nối các docker host lại với nhau

<b>Các ổ đĩa - volume trên swarm</b><br>
Để chia sẻ dữ liệu giữa các container chạy trên node khác nhau cần dùng đến các loại ổ đĩa mạng bằng các giao thức như nfs, smb, ssh ...

<b>Swarm có nhiều Node Manager</b><br>
Ở trên tạo ra một Swarm với 1 node vài trò manager (vps1) và 2 node worker (vps3, vps2). Trong tình huống này, nếu node manager bị lỗi dẫn tới toàn bộ swarm không hoạt động. Trong tình huống đề phòng, có thể đề xuất vps2, vps3 cũng có vai trò Manager, nếu node manager chính (vps1) chết, hệ thống tự động kích hoạt vps2 là node manager ... <br>

Để làm điều đó, thực hiện lệnh tại node manager đang hoạt động, lệnh này là đề xuất vps2 sẽ là node manager nếu vps1 die. <br>

Tại vps1, gõ lệnh <br>

    docker node promote NODEID
    
Từ giờ thì node vps2 cũng có vai trò Manager, nó sẽ thay thế cho vps1 nếu vps1 không còn hoạt động